In [21]:
import deepspeed
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision.transforms import ToTensor

In [20]:
# Download training data from open datasets.
batch_size = 64
datasets_list = [
    (
     "FashionMNIST",
     datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor()),
     datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())
    ),
    (
     "CIFAR10",
     datasets.CIFAR10(root="data", train=True, download=True, transform=ToTensor()),
     datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
    ),
    (
     "CIFAR100",
     datasets.CIFAR100(root="data", train=True, download=True, transform=ToTensor()),
     datasets.CIFAR100(root="data", train=False, download=True, transform=ToTensor())
    ),
    (
     "MNIST",
     datasets.MNIST(root="data", train=True, download=True, transform=ToTensor()),
     datasets.MNIST(root="data", train=False, download=True, transform=ToTensor())
    ),
    (
     "SVHN",
     datasets.SVHN(root="data", split="train", download=True, transform=ToTensor()),
     datasets.SVHN(root="data", split="test", download=True, transform=ToTensor())
    )
]

dataloaders_list = []
for dataset in datasets_list:
    dataloaders_list.append(
        (
            # Dataset Name
            dataset[0],

            # Training Dataloader
            DataLoader(dataset[1], batch_size=batch_size),

            # Testing Dataloader
            DataLoader(dataset[2], batch_size=batch_size)
        )
    )

for dataloader in dataloaders_list:
    for X, y in dataloader[2]:
        print(f"Dataset: {dataloader[0]}")
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 182040794/182040794 [01:01<00:00, 2960470.75it/s] 


100%|██████████| 64275384/64275384 [00:08<00:00, 7504093.03it/s]


Dataset: FashionMNIST
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Dataset: CIFAR10
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64
Dataset: CIFAR100
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64
Dataset: MNIST
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Dataset: SVHN
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [22]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

models_list = [
    (
        "AlexNet",
        models.alexnet(pretrained=False)
    ),
    (
        "VGG16",
        models.vgg16(pretrained=False)
    ),
    (
        "Inception",
        models.inception_v3(pretrained=False)
    ),
    (
        "ResNet50",
        models.resnet50(pretrained=False)
    )
]

loss_fn_list = [
    nn.CrossEntropyLoss()
]

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


/home/andrew/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/andrew/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/andrew/.local/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [15]:
# DeepSpeed Configuration
ds_config = {
    "train_micro_batch_size_per_gpu": 64,
    "optimizer": {
        "type": "SGD",
        "params": {
            "lr": 0.001
        }
    },
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 2     
    },
    "zero_allow_untested_optimizer": True,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "wall_clock_breakdown": False
}

model_engine, _, _, _ = deepspeed.initialize(args=None,
                                                      model=model,
                                                      model_parameters=model.parameters(),
                                                      config_params=ds_config)

def train(dataloader, model_engine, loss_fn):
    size = len(dataloader.dataset)
    model_engine.train()

    for batch, (X, y) in enumerate(dataloader):
        # Ensure data is on the correct device
        X, y = X.to(model_engine.local_rank).half(), y.to(model_engine.local_rank)

        # Forward pass using model_engine
        pred = model_engine(X)
        loss = loss_fn(pred, y)

        # Backpropagation and weight update
        model_engine.backward(loss)
        model_engine.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # Ensure data is on the correct device
            X, y = X.to(model_engine.local_rank).half(), y.to(model_engine.local_rank)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Training and Testing Model
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_engine, loss_fn)
    test(test_dataloader, model, loss_fn)
print("Done!")

[2023-11-20 16:03:16,988] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.12.3, git-hash=unknown, git-branch=unknown
[2023-11-20 16:03:16,994] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-11-20 16:03:16,995] [INFO] [logging.py:96:log_dist] [Rank 0] Using DeepSpeed Optimizer param name SGD as basic optimizer
[2023-11-20 16:03:16,996] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2023-11-20 16:03:16,997] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = SGD
[2023-11-20 16:03:16,998] [INFO] [utils.py:56:is_zero_supported_optimizer] Checking ZeRO support for optimizer=SGD type=<class 'torch.optim.sgd.SGD'>
[2023-11-20 16:03:16,999] [WARNING] [engine.py:1160:_do_optimizer_sanity_check] **** You are using ZeRO with an untested optimizer, proceed with caution *****
[2023-11-20 16:03:17,000] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.float16 

In [ ]:
# Saving Model

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
# Loading Model

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [14]:
# Inference

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Ankle boot"
